# Calcular el IC

- El valor del IC no coincidía con los valores de goatools para los mismos términos.

$$
IC = -\log\left(\frac{tcnts}{total}\right)
$$

A partir de las anotaciones .gaf.

En estas anotaciones, cada fila tiene: **ID de la anotacion** ("P12345"), **GO Term**, **Evidence** ("IEA").

## ¿Cómo lo hace goatools?

- Las anotaciones **se propagan** hacia los ancestros. Si un término está anotado con una cierta anotacion (P12345), todos sus ancestros se anotan también.

        Supongamos que el término A tiene ancestros B y C:
        •	Si tienes una anotación en A, entonces:
        •	tcnt[A] += 1
        •	tcnt[B] += 1
        •	tcnt[C] += 1

- No se divide por el total de anotaciones, si no por el **total del namespace** correspondiente: BP, MF o CC.

        Cómo funciona esto con la propagación de anotaciones? Yo he contado cada anotación propagada como una nueva anotación tanto para el counts como para el total.

- Aun así no me daba el mismo IC.

## Problemas que había.

- Los archivos de anotación GAF tienen filas con IDs de anotación repetidos.

        Supongamos que tenemos el db_object_id = "P12345" con dos anotaciones:
            •	GO:0006397 (con ancestros A, B, C)
            •	GO:0006396 (que también comparte B, C)

        El código actual suma así:
            •	B: 2 veces (una por cada anotación)
            •	C: 2 veces

        Pero Goatools sólo cuenta:
            •	B: 1 vez para ese objeto
            •	C: 1 vez


- Primero hay que agrupar por ``db_object_id``. Luego acumular los términos + ancestros y contar una única vez por objeto.

- Para el total por namespace, cada ``db_object_id`` cuenta una única vez para el ``total_by_ns``.


In [1]:
import go3

go3.load_go_terms()
annots = go3.load_gaf("../src/goa_human.gaf")
tc_rust = go3.build_term_counter(annots)

In [8]:
from goatools.obo_parser import GODag
from goatools.associations import dnld_assc
from goatools.semantic import TermCounts, resnik_sim

obo_path = "go-basic.obo"
gaf_path = "goa_human.gaf"
godag = GODag(obo_path)
assocs = dnld_assc(gaf_path, godag)
termcounts = TermCounts(godag, assocs)

go-basic.obo: fmt(1.2) rel(2025-03-16) 43,544 Terms
HMS:0:00:10.880900 969,176 annotations READ: goa_human.gaf 
36411 IDs in loaded association branch, BP


In [5]:
go_id_1, go_id_2 = "GO:0006397", "GO:0008380"
print(go3.term_ic(go_id_1, tc_rust))
print(go3.term_ic(go_id_2, tc_rust))
resnik = go3.semantic_similarity(go_id_1, go_id_2, 'lin', tc_rust)
print(resnik)
print("Counts:", tc_rust.counts[go_id_1])
#print("Total by namespace:", tc_rust.total_by_ns["BP"])
print("IC:", tc_rust.ic[go_id_1])

sim = go3.compare_genes(
    "WDFY1",
    "TFAM",
    ontology="MF",
    similarity="resnik",
    groupwise="max",
    counter=tc_rust
)
print(sim)

4.272783893072207
4.570337714003141
0.4734022960071144
Counts: 509
IC: 4.272783893072207
0.2948069841169292


In [5]:
pairs = [("WDFY1", "TFAM"), ("Q8N158", "Q8N158"), ("P12345", "Q8N158")]

sims = go3.compare_gene_pairs_batch(
    pairs,
    ontology="MF",
    method="resnik",
    combine="bma",
    counter=tc_rust
)

print(sims)

[0.2852970814034799, 0.0, 0.0]


In [10]:
from goatools.semantic import get_info_content

print(get_info_content(go_id_1, termcounts=termcounts))
print(get_info_content(go_id_2, termcounts=termcounts))
resnik_sim(go_id_1, go_id_2, termcounts=termcounts, godag=godag)
print(f"Counts for {go_id_1}: {termcounts.get_count(go_id_1)}")

4.270178189223948
4.5677320101548835
Counts for GO:0006397: 509
